In [1]:
import pandas as pd
import requests
import json

In [2]:
meteoriteFile = "Resources/meteorite-landings.csv"

In [3]:
meteoriteDF = pd.read_csv(meteoriteFile, encoding="ISO-8859-1")

In [12]:
meteoriteDF.shape

(45716, 10)

In [14]:
meteoriteDF.columns

Index(['name', 'id', 'nametype', 'recclass', 'mass', 'fall', 'year', 'reclat',
       'reclong', 'GeoLocation'],
      dtype='object')

In [16]:
meteoriteDF.dtypes

name            object
id               int64
nametype        object
recclass        object
mass           float64
fall            object
year           float64
reclat         float64
reclong        float64
GeoLocation     object
dtype: object

In [4]:
meteoriteDF.head()

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775000, 6.083330)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.183330, 10.233330)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.216670, -113.000000)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.883330, -99.900000)"
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.166670, -64.950000)"


In [13]:
meteoriteDF.describe() 

,id,mass,year,reclat,reclong
count,45716.000000,4.558500e+04,45428.000000,38401.000000,38401.000000
mean,26889.735104,1.327808e+04,1991.772189,-39.122580,61.074319
std,16860.683030,5.749889e+05,27.181247,46.378511,80.647298
min,1.000000,0.000000e+00,301.000000,-87.366670,-165.433330
25%,12688.750000,7.200000e+00,1987.000000,-76.714240,0.000000
50%,24261.500000,3.260000e+01,1998.000000,-71.500000,35.666670
75%,40656.750000,2.026000e+02,2003.000000,0.000000,157.166670
max,57458.000000,6.000000e+07,2501.000000,81.166670,354.473330


In [15]:
meteoriteDF.isnull().sum()

name              0
id                0
nametype          0
recclass          0
mass            131
fall              0
year            288
reclat         7315
reclong        7315
GeoLocation    7315
dtype: int64

In [6]:
meteoriteFellDF = meteoriteDF[meteoriteDF["fall"] == "Fell"]

In [17]:
meteoriteFellDF.isnull().sum()

name            0
id              0
nametype        0
recclass        0
mass           32
fall            0
year            0
reclat         10
reclong        10
GeoLocation    10
dtype: int64

In [7]:
meteoriteFellDF = meteoriteFellDF.drop(meteoriteFellDF[(meteoriteFellDF["reclat"] == 0) & (meteoriteFellDF["reclong"] == 0)].index)

In [8]:
meteoriteFellDF = meteoriteFellDF.drop(meteoriteFellDF[(meteoriteFellDF["year"] >= 2016.0) | (meteoriteFellDF["year"] < 860.0)].index)

In [18]:
locationDF = meteoriteFellDF[['id', 'reclat', 'reclong']]

In [19]:
locationDF["city"] = ""
locationDF["country"] = ""

/Users/mario/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/mario/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
locationDF.head()

,id,reclat,reclong,city,country
0,1,50.77500,6.08333,,
1,2,56.18333,10.23333,,
2,6,54.21667,-113.00000,,
3,10,16.88333,-99.90000,,
4,370,-33.16667,-64.95000,,


In [21]:
params = {"key": "value"}

In [22]:
for index,row in locationDF.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    lat = row['reclat']
    lng = row['reclong']
    
    params['latlng'] = f"{lat},{lng}"
    
    meteoriteCityCountry = requests.get(base_url,params=params)
    
    meteoriteCityCountry = meteoriteCityCountry.json()
    
    try:
        if meteoriteCityCountry["results"][0]["address_components"][0]["types"][0] == "locality":
            locationDF.loc[index, "city"] = meteoriteCityCountry["results"][0]["address_components"][0]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][1]["types"][0] == "locality":
            locationDF.loc[index, "city"] = meteoriteCityCountry["results"][0]["address_components"][1]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][2]["types"][0] == "locality":
            locationDF.loc[index, "city"] = meteoriteCityCountry["results"][0]["address_components"][2]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][3]["types"][0] == "locality":
            locationDF.loc[index, "city"] = meteoriteCityCountry["results"][0]["address_components"][3]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][4]["types"][0] == "locality":
            locationDF.loc[index, "city"] = meteoriteCityCountry["results"][0]["address_components"][4]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][5]["types"][0] == "locality":
            locationDF.loc[index, "city"] = meteoriteCityCountry["results"][0]["address_components"][5]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][6]["types"][0] == "locality":
            locationDF.loc[index, "city"] = meteoriteCityCountry["results"][0]["address_components"][6]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][7]["types"][0] == "locality":
            locationDF.loc[index, "city"] = meteoriteCityCountry["results"][0]["address_components"][7]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][8]["types"][0] == "locality":
            locationDF.loc[index, "city"] = meteoriteCityCountry["results"][0]["address_components"][8]["long_name"]
        else:
            pass
    except:
        locationDF.loc[index, "city"] = ""
    
    try:
        if meteoriteCityCountry["results"][0]["address_components"][0]["types"][0] == "country":
            locationDF.loc[index, "country"] = meteoriteCityCountry["results"][0]["address_components"][0]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][1]["types"][0] == "country":
            locationDF.loc[index, "country"] = meteoriteCityCountry["results"][0]["address_components"][1]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][2]["types"][0] == "country":
            locationDF.loc[index, "country"] = meteoriteCityCountry["results"][0]["address_components"][2]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][3]["types"][0] == "country":
            locationDF.loc[index, "country"] = meteoriteCityCountry["results"][0]["address_components"][3]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][4]["types"][0] == "country":
            locationDF.loc[index, "country"] = meteoriteCityCountry["results"][0]["address_components"][4]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][5]["types"][0] == "country":
            locationDF.loc[index, "country"] = meteoriteCityCountry["results"][0]["address_components"][5]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][6]["types"][0] == "country":
            locationDF.loc[index, "country"] = meteoriteCityCountry["results"][0]["address_components"][6]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][7]["types"][0] == "country":
            locationDF.loc[index, "country"] = meteoriteCityCountry["results"][0]["address_components"][7]["long_name"]
        elif meteoriteCityCountry["results"][0]["address_components"][8]["types"][0] == "country":
            locationDF.loc[index, "country"] = meteoriteCityCountry["results"][0]["address_components"][8]["long_name"]
        else:
            pass
    except:
        locationDF.loc[index, "country"] = ""

/Users/mario/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [23]:
locationDF.tail(20)

,id,reclat,reclong,city,country
1088,30377,28.71667,116.61667,Shangrao Shi,China
1089,30378,47.31667,35.36667,,Ukraine
1090,30379,50.28333,27.68333,Baranivka,Ukraine
1091,30380,55.18333,27.91667,,Belarus
1092,30384,27.33333,-9.33333,,
1093,30386,11.73333,7.08333,,Nigeria
1094,30388,47.50000,83.00000,Oichilik,Kazakhstan
1095,30389,33.70000,133.80000,Kami,Japan
1096,30391,32.30000,112.75000,Xiangyang Shi,China
1097,30395,47.13333,43.90000,,Russia


In [ ]:
locationDF[locationDF["country"] == ""]

In [24]:
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
params['latlng'] = "27.33333,-9.33333"
    
response = requests.get(base_url, params=params)
locationData = response.json()
print(json.dumps(locationData, indent=4, sort_keys=True))

{
    "plus_code": {
        "global_code": "7CVG8MM8+8M"
    },
    "results": [],
    "status": "ZERO_RESULTS"
}
